# Práctica 2

**Datos**

**Alumno:** Alan Ruiz Mondragón  
**Grupo:** 18

In [10]:
from datetime import datetime
from textblob import TextBlob, Word, Blobber
from textblob.classifiers import NaiveBayesClassifier
from textblob.taggers import NLTKTagger
import emoji
import jellyfish as jf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import regex
import seaborn as sns
import unicodedata

In [13]:
df=pd.read_table('wa_chat.txt', delimiter='\t')
df=df.reset_index().T.reset_index().T
df

FileNotFoundError: [Errno 2] No such file or directory: 'wa_chat.txt'

### Parte 2 - Ingeniería continuas

**Objetivo**

Predecir el valor al cierre de las acciones de Facebook de la siguiente semana, utilizando técnicas de ingeniería de variables que nos permita construir una TAD para generar columnas que aporten valor a nuestro objetivo.

#### Librerias

In [7]:
from datetime import datetime
#import emoji
import jellyfish as jf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import unicodedata
import re

#### Dataset

In [13]:
fb = pd.read_csv('/work/files/workspace/practica_2/data/[M1] - FB stocks.csv')

In [10]:
fb.head()

,Fecha,Cierre,Apertura,Máximo,Mínimo,Quarter,Weekday
0,2012-01-06,27.72,28.89,29.15,27.39,1,Friday
1,2012-01-08,20.88,21.50,21.58,20.84,1,Sunday
2,2012-01-10,21.99,22.08,22.59,21.73,1,Tuesday
3,2012-01-11,21.21,21.08,21.44,21.01,1,Wednesday
4,2012-02-07,30.77,31.25,31.73,30.55,1,Tuesday


In [ ]:
fb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1415 entries, 0 to 1414
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Fecha     1415 non-null   object 
 1   Cierre    1415 non-null   float64
 2   Apertura  1415 non-null   float64
 3   Máximo    1415 non-null   float64
 4   Mínimo    1415 non-null   float64
 5   Quarter   1415 non-null   int64  
 6   Weekday   1415 non-null   object 
dtypes: float64(4), int64(1), object(2)
memory usage: 77.5+ KB


Modificamos la variable de fecha a un tipo datetime para su posterior manipulación

In [ ]:
fb['Fecha'] = pd.to_datetime(fb['Fecha'])

In [ ]:
fb.shape

(1415, 7)

#### Etiquetado de variables

**Antes del etiquetado de variables realizamos modificaciones en el nombre de las columnas**

In [ ]:
fb = fb.rename(columns=str.lower)
fb = fb.rename(columns={'máximo': 'maximo', 'mínimo':'minimo'})
fb.head()

,fecha,cierre,apertura,maximo,minimo,quarter,weekday
0,2012-01-06,27.72,28.89,29.15,27.39,1,Friday
1,2012-01-08,20.88,21.50,21.58,20.84,1,Sunday
2,2012-01-10,21.99,22.08,22.59,21.73,1,Tuesday
3,2012-01-11,21.21,21.08,21.44,21.01,1,Wednesday
4,2012-02-07,30.77,31.25,31.73,30.55,1,Tuesday


**Realizamos el etiquetado de las variables**

Los prefijos que utilizaremos para los tipos de variables se describen de la siguiente manera:

**v_**: Variables categoricas.  
**c_**: Variables numericas (discretas y continuas).   
**d_**: Variables tipo fecha.  
**t_**: Variables de texto (comentarios, descripciones, url, etc.).  
**g_**: Variables geograficas.  


In [ ]:
v_feats=['quarter', 'weekday']
c_feats=['cierre','apertura',"maximo", "minimo"]
d_feats=['fecha']

v_feats_new=["v_"+x for x in v_feats]
c_feats_new=["c_"+x for x in c_feats]
d_feats_new=["d_"+x for x in d_feats]

fb.rename(columns=dict(zip(v_feats,v_feats_new)),inplace=True)
fb.rename(columns=dict(zip(c_feats,c_feats_new)),inplace=True)
fb.rename(columns=dict(zip(d_feats,d_feats_new)),inplace=True)

In [ ]:
fb.head()

,d_fecha,c_cierre,c_apertura,c_maximo,c_minimo,v_quarter,v_weekday
0,2012-01-06,27.72,28.89,29.15,27.39,1,Friday
1,2012-01-08,20.88,21.50,21.58,20.84,1,Sunday
2,2012-01-10,21.99,22.08,22.59,21.73,1,Tuesday
3,2012-01-11,21.21,21.08,21.44,21.01,1,Wednesday
4,2012-02-07,30.77,31.25,31.73,30.55,1,Tuesday


In [ ]:
fb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1415 entries, 0 to 1414
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   d_fecha     1415 non-null   datetime64[ns]
 1   c_cierre    1415 non-null   float64       
 2   c_apertura  1415 non-null   float64       
 3   c_maximo    1415 non-null   float64       
 4   c_minimo    1415 non-null   float64       
 5   v_quarter   1415 non-null   int64         
 6   v_weekday   1415 non-null   object        
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 77.5+ KB


In [ ]:
fb.shape

(1415, 7)

#### Calidad de datos

##### Duplicados

In [ ]:
#Total de registros duplicados de forma general
fb.duplicated().sum()

0

In [ ]:
#Total de registros duplicados por la variable de fecha
fb.duplicated(subset = 'd_fecha').sum()

0

In [ ]:
fb.head()

,d_fecha,c_cierre,c_apertura,c_maximo,c_minimo,v_quarter,v_weekday
0,2012-01-06,27.72,28.89,29.15,27.39,1,Friday
1,2012-01-08,20.88,21.50,21.58,20.84,1,Sunday
2,2012-01-10,21.99,22.08,22.59,21.73,1,Tuesday
3,2012-01-11,21.21,21.08,21.44,21.01,1,Wednesday
4,2012-02-07,30.77,31.25,31.73,30.55,1,Tuesday


In [ ]:
fb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1415 entries, 0 to 1414
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   d_fecha     1415 non-null   datetime64[ns]
 1   c_cierre    1415 non-null   float64       
 2   c_apertura  1415 non-null   float64       
 3   c_maximo    1415 non-null   float64       
 4   c_minimo    1415 non-null   float64       
 5   v_quarter   1415 non-null   int64         
 6   v_weekday   1415 non-null   object        
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 77.5+ KB


In [ ]:
fb.shape

(1415, 7)

##### Completitud

In [ ]:
def completitud(df):
    comple = pd.DataFrame(df.isnull().sum())
    comple.reset_index(inplace = True)
    comple = comple.rename(columns = {'index': 'columna', 0: 'nulos'})
    comple['completitud'] = (1-comple['nulos']/df.shape[0])*100
    comple = comple.sort_values(by = 'completitud', ascending = True)
    comple.reset_index(drop = True, inplace = True)
    return comple

completitud(fb)

,columna,nulos,completitud
0,d_fecha,0,100.0
1,c_cierre,0,100.0
2,c_apertura,0,100.0
3,c_maximo,0,100.0
4,c_minimo,0,100.0
5,v_quarter,0,100.0
6,v_weekday,0,100.0


In [ ]:
fb.head()

,d_fecha,c_cierre,c_apertura,c_maximo,c_minimo,v_quarter,v_weekday
0,2012-01-06,27.72,28.89,29.15,27.39,1,Friday
1,2012-01-08,20.88,21.50,21.58,20.84,1,Sunday
2,2012-01-10,21.99,22.08,22.59,21.73,1,Tuesday
3,2012-01-11,21.21,21.08,21.44,21.01,1,Wednesday
4,2012-02-07,30.77,31.25,31.73,30.55,1,Tuesday


In [ ]:
fb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1415 entries, 0 to 1414
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   d_fecha     1415 non-null   datetime64[ns]
 1   c_cierre    1415 non-null   float64       
 2   c_apertura  1415 non-null   float64       
 3   c_maximo    1415 non-null   float64       
 4   c_minimo    1415 non-null   float64       
 5   v_quarter   1415 non-null   int64         
 6   v_weekday   1415 non-null   object        
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 77.5+ KB


In [ ]:
fb.shape

(1415, 7)

##### Consistencia y precisión

###### Variables categoricas

In [ ]:
#Revisamos las variables categoricas
for i in fb.filter(like = 'v_'):
    print(i)
    values = fb[i].astype(str).unique()
    values.sort()
    display(values)
    print('\n')

v_quarter


array(['1', '2', '3', '4'], dtype=object)



v_weekday


array(['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday',
       'Wednesday'], dtype=object)

**v_weekday**

Nos aseguramos de tomar los días correctos

In [ ]:
fb['v_weekday'] = fb['d_fecha'].map(lambda x: x.strftime('%A'))

Debido a que la naturaleza de los datos corresponden al precio de las acciones de **Facebook**, podemos inferir en que el los datos podrían ser inconsistentes al presentar datos en días de fin de semana, pues los **mercados financieros** solo funcionan y fijan precios de sus instrumentos de renta variable en los días de lunes a viernes.

En este sentido es viable eliminar estos registros y quedarnos con los que corresponden a los registros que deberían ser verídicos.

In [ ]:
#Modificaciones en la varible de días de la semana
fb['v_weekday'] = fb['v_weekday'].str.lower()
fb = fb.astype(dtype = {'v_weekday': 'string'})

#Eliminamos los datos fin de semana
fb = fb.drop(fb[fb['v_weekday'] == 'saturday'].index)
fb = fb.drop(fb[fb['v_weekday'] == 'sunday'].index)
fb.reset_index(drop = True, inplace = True)

In [ ]:
fb['v_weekday'].describe()

count       1265
unique         5
top       friday
freq         262
Name: v_weekday, dtype: object

In [ ]:
fb['v_weekday'].value_counts()

friday       262
wednesday    258
tuesday      256
thursday     250
monday       239
Name: v_weekday, dtype: Int64

In [ ]:
fb.head()

,d_fecha,c_cierre,c_apertura,c_maximo,c_minimo,v_quarter,v_weekday
0,2012-01-06,27.72,28.89,29.15,27.39,1,friday
1,2012-01-10,21.99,22.08,22.59,21.73,1,tuesday
2,2012-01-11,21.21,21.08,21.44,21.01,1,wednesday
3,2012-02-07,30.77,31.25,31.73,30.55,1,tuesday
4,2012-02-08,20.04,20.77,20.84,19.82,1,wednesday


Ahora que hemos quitado los días correspondientes a los fines de semana, podemos observar que los días no son continuos y que nos falta información que podría ser de ayuda en futuros modelos, de modo que más adelante, el término de la revisión de calidad de datos haremos modificaciones para duplicar valores del periodo anterior para los casos que no tenemos

In [ ]:
fb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1265 entries, 0 to 1264
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   d_fecha     1265 non-null   datetime64[ns]
 1   c_cierre    1265 non-null   float64       
 2   c_apertura  1265 non-null   float64       
 3   c_maximo    1265 non-null   float64       
 4   c_minimo    1265 non-null   float64       
 5   v_quarter   1265 non-null   int64         
 6   v_weekday   1265 non-null   string        
dtypes: datetime64[ns](1), float64(4), int64(1), string(1)
memory usage: 69.3 KB


In [ ]:
fb.shape

(1265, 7)

**v_quarter**

In [ ]:
fb['v_quarter'].value_counts()

3    340
4    333
2    310
1    282
Name: v_quarter, dtype: int64

Revisamos nuevamente

In [ ]:
#Revisamos de forma general las variables categoricas
for i in fb.filter(like="v_"):
    print(i)
    values = fb[i].astype(str).unique()
    values.sort()
    display(values)
    print("\n")

v_quarter


array(['1', '2', '3', '4'], dtype=object)



v_weekday


array(['friday', 'monday', 'thursday', 'tuesday', 'wednesday'],
      dtype=object)

In [ ]:
fb.head()

,d_fecha,c_cierre,c_apertura,c_maximo,c_minimo,v_quarter,v_weekday
0,2012-01-06,27.72,28.89,29.15,27.39,1,friday
1,2012-01-10,21.99,22.08,22.59,21.73,1,tuesday
2,2012-01-11,21.21,21.08,21.44,21.01,1,wednesday
3,2012-02-07,30.77,31.25,31.73,30.55,1,tuesday
4,2012-02-08,20.04,20.77,20.84,19.82,1,wednesday


In [ ]:
fb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1265 entries, 0 to 1264
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   d_fecha     1265 non-null   datetime64[ns]
 1   c_cierre    1265 non-null   float64       
 2   c_apertura  1265 non-null   float64       
 3   c_maximo    1265 non-null   float64       
 4   c_minimo    1265 non-null   float64       
 5   v_quarter   1265 non-null   int64         
 6   v_weekday   1265 non-null   string        
dtypes: datetime64[ns](1), float64(4), int64(1), string(1)
memory usage: 69.3 KB


In [ ]:
fb.shape

(1265, 7)

###### Variables númericas

In [ ]:
#Revisamos las variables continuas
for i in fb.filter(like= 'c_'):
    print(i)
    values = fb[i].astype(str).unique()
    values.sort()
    display(values)
    print("\n")

c_cierre


array(['101.61', '101.91', '101.97', ..., '99.54', '99.67', '99.75'],
      dtype=object)



c_apertura


array(['100.41', '100.5', '100.58', ..., '99.0', '99.6', '99.88'],
      dtype=object)



c_maximo


array(['100.5', '100.58', '101.43', ..., '99.59', '99.73', '99.96'],
      dtype=object)



c_minimo


array(['100.16', '100.24', '100.4', ..., '97.55', '98.88', '99.75'],
      dtype=object)

In [ ]:
fb.describe()

,c_cierre,c_apertura,c_maximo,c_minimo,v_quarter
count,1265.000000,1265.000000,1265.000000,1265.000000,1265.000000
mean,85.359779,85.367478,86.222379,84.431787,2.572332
std,45.661868,45.634000,45.830568,45.396818,1.103659
min,17.730000,18.080000,18.270000,17.550000,1.000000
25%,47.010000,46.990000,47.550000,46.260000,2.000000
50%,79.600000,79.540000,80.380000,78.630000,3.000000
75%,118.810000,118.920000,119.610000,117.670000,4.000000
max,183.030000,183.510000,184.250000,182.090000,4.000000


In [ ]:
#Revisamos si las variables continuas tienen datos no correspodientes a su naturaleza
for i in fb.filter(like = 'c_'):
    print(i)
    values = fb[fb[i].map(lambda x: sum([y.isalpha() for y in str(x)])>0)]
    values
    print("\n")

c_cierre


c_apertura


c_maximo


c_minimo




In [ ]:
fb.head()

,d_fecha,c_cierre,c_apertura,c_maximo,c_minimo,v_quarter,v_weekday
0,2012-01-06,27.72,28.89,29.15,27.39,1,friday
1,2012-01-10,21.99,22.08,22.59,21.73,1,tuesday
2,2012-01-11,21.21,21.08,21.44,21.01,1,wednesday
3,2012-02-07,30.77,31.25,31.73,30.55,1,tuesday
4,2012-02-08,20.04,20.77,20.84,19.82,1,wednesday


In [ ]:
fb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1265 entries, 0 to 1264
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   d_fecha     1265 non-null   datetime64[ns]
 1   c_cierre    1265 non-null   float64       
 2   c_apertura  1265 non-null   float64       
 3   c_maximo    1265 non-null   float64       
 4   c_minimo    1265 non-null   float64       
 5   v_quarter   1265 non-null   int64         
 6   v_weekday   1265 non-null   string        
dtypes: datetime64[ns](1), float64(4), int64(1), string(1)
memory usage: 69.3 KB


In [ ]:
fb.shape

(1265, 7)

###### Variables de fecha

In [ ]:
##Revisamos las variables fecha
for i in fb.filter(like="d_"):
    print(i)
    values = fb[i].astype(str).unique()
    values.sort()
    display(values)
    print("\n")

d_fecha


array(['2012-01-06', '2012-01-10', '2012-01-11', ..., '2017-12-27',
       '2017-12-28', '2017-12-29'], dtype=object)

In [ ]:
fb['d_fecha'].value_counts()

2012-01-06    1
2016-02-08    1
2016-02-19    1
2016-02-18    1
2016-02-17    1
             ..
2014-04-02    1
2014-03-31    1
2014-03-28    1
2014-03-27    1
2017-12-29    1
Name: d_fecha, Length: 1265, dtype: int64

In [ ]:
fb['d_fecha'].describe()

count                    1265
unique                   1265
top       2012-01-06 00:00:00
freq                        1
first     2012-01-06 00:00:00
last      2017-12-29 00:00:00
Name: d_fecha, dtype: object

Revisamos la frecuencia de registros mediante la fecha y podemos observar que hay una gran disparidad entre algunos registros, incluso en diferencias de registro de casi un mes

In [ ]:
fb["d_fecha"].diff().describe()

count                         1264
mean     1 days 17:28:06.075949367
std      1 days 17:58:11.332996044
min                1 days 00:00:00
25%                1 days 00:00:00
50%                1 days 00:00:00
75%                2 days 00:00:00
max               27 days 00:00:00
Name: d_fecha, dtype: object

Agregamos el número de semana para poder encontrar las mayores distancias entre registros. Esto lo hacemos revisando cada año con ayuda de una nueva columna que representa el número de semana.

In [ ]:
fb['v_week_number'] = fb["d_fecha"].dt.week

Como se observa a continuación, el año 2012 es el peor registrado, pues de las 52 semanas que hubo en este año, solo se tienen registros de 40, sin considerar faltantes por semana.

Las semanas faltantes:

**11** se encuentran entre la semana **1 a 19**  
**1** se encuentra entre la 39 y 41

In [ ]:
anio = [2012, 2013, 2014, 2015, 2016, 2017]
for i in anio:
    print(f'Año: {i}')
    values = fb[fb['d_fecha'].map(lambda x: x.year == i)].reset_index(drop = True)
    values = values['v_week_number'].unique()
    print(f'Semanas registradas: {len(values)}')
    values.sort()
    display(values)
    print("\n")

Año: 2012
Semanas registradas: 40


array([ 1,  2,  6, 10, 11, 14, 15, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28,
       29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46,
       47, 48, 49, 50, 51, 52])



Año: 2013
Semanas registradas: 52


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52])



Año: 2014
Semanas registradas: 52


...

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53])



Año: 2016
Semanas registradas: 52


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52])



Año: 2017
Semanas registradas: 52


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52])

Una opción es eliminar todos los datos del año 2012 donde no existe una regularidad en los registros, es decir, hasta la semana 19, pero también es importante ver no solo cuantas semanas hubo registros, sino también si esas semanas fueron completas o con un porcentaje aceptable.

Aqui definimos una regla:

Si suponemos que 1 semana entera de registro de los precios de las acciones de Facebook son de 5 registros, es decir, los correspondientes a cada día de Lunes a Viernes y un mes tiene 4 semanas, podemos decir que el número de registros en un mes es de 20, de modo que si tenemos un registro perdido por semana estamos diciendo que podemos aceptar la perdida de 5 registros al mes, es decir, tener 15 registros mensuales, lo que prespresenta un 75%, de modo que los meses que representen un porcentaje menor a este, no serán considerados.

Con base en lo anterior decidimos que registros eliminaremos para cada año si así se requiere.

Revisión año **2012**

In [ ]:
def faltante(df):
    reg = pd.DataFrame(fb['d_fecha'].map(lambda x: x.month)[fb["d_fecha"].map(lambda x: x.year == 2012)]
                       .value_counts())
    reg.reset_index(inplace=True)
    reg=reg.rename(columns={"index":"mes",'d_fecha':"registros_totales"})
    reg["proporcion"]=(reg["registros_totales"]/20)*100
    reg=reg.sort_values(by="mes",ascending=True)
    reg.reset_index(drop=True,inplace=True)
    return reg

faltante(fb)

,mes,registros_totales,proporcion
0,1,3,15.0
1,2,3,15.0
2,3,3,15.0
3,4,4,20.0
4,5,13,65.0
5,6,18,90.0
6,7,17,85.0
7,8,18,90.0
8,9,15,75.0
9,10,15,75.0


Obtenemos el porcentaje que representan estos elementos sobre el total, de ser bajo procedemos a eliminar los registros correspondientes a los meses de enero a mayo, ya que contienen < 75% de los datos mensuales.

In [ ]:
(fb['d_fecha'].map(lambda x: x.month)[(fb['d_fecha'].map(lambda x: x.month <= 5)) & 
                                      (fb['d_fecha'].map(lambda x: x.year == 2012))].value_counts().sum()
 /fb.shape[0]) * 100

2.0553359683794468

Dado que el porcentaje de datos perdidos si consideramos eliminar la ventana de registros con una mayor cantidad de (meses enero a mayo del año 2012) **missings**, entonces eliminamos.

In [ ]:
fb = fb[fb['d_fecha'] >= '2012-06-01']
fb = fb.reset_index(drop=True)
fb = fb.drop('v_week_number', axis=1)
fb.head()

,d_fecha,c_cierre,c_apertura,c_maximo,c_minimo,v_quarter,v_weekday
0,2012-06-06,26.81,26.07,27.17,25.52,2,wednesday
1,2012-06-07,31.73,31.44,31.90,31.26,2,thursday
2,2012-06-08,21.92,21.39,22.15,21.30,2,friday
3,2012-06-11,21.17,21.24,21.37,20.99,2,monday
4,2012-06-12,26.97,27.68,27.75,26.82,2,tuesday


In [ ]:
fb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1239 entries, 0 to 1238
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   d_fecha     1239 non-null   datetime64[ns]
 1   c_cierre    1239 non-null   float64       
 2   c_apertura  1239 non-null   float64       
 3   c_maximo    1239 non-null   float64       
 4   c_minimo    1239 non-null   float64       
 5   v_quarter   1239 non-null   int64         
 6   v_weekday   1239 non-null   string        
dtypes: datetime64[ns](1), float64(4), int64(1), string(1)
memory usage: 67.9 KB


In [ ]:
fb['d_fecha'].describe()

count                    1239
unique                   1239
top       2012-06-06 00:00:00
freq                        1
first     2012-06-06 00:00:00
last      2017-12-29 00:00:00
Name: d_fecha, dtype: object

In [ ]:
fb["d_fecha"].diff().describe()

count                         1238
mean     1 days 15:23:33.247172859
std      1 days 02:10:10.686775038
min                1 days 00:00:00
25%                1 days 00:00:00
50%                1 days 00:00:00
75%                2 days 00:00:00
max               10 days 00:00:00
Name: d_fecha, dtype: object

In [ ]:
df = pd.DataFrame(pd.date_range(start='2012-06-06', end='2017-12-29'), columns=['d_fecha'])
df.head()

,d_fecha
0,2012-06-06
1,2012-06-07
2,2012-06-08
3,2012-06-09
4,2012-06-10


In [ ]:
fb = df.merge(fb, on='d_fecha', how='left')
fb

,d_fecha,c_cierre,c_apertura,c_maximo,c_minimo,v_quarter,v_weekday
0,2012-06-06,26.81,26.07,27.17,25.52,2.0,wednesday
1,2012-06-07,31.73,31.44,31.90,31.26,2.0,thursday
2,2012-06-08,21.92,21.39,22.15,21.30,2.0,friday
3,2012-06-09,NaN,NaN,NaN,NaN,NaN,<NA>
4,2012-06-10,NaN,NaN,NaN,NaN,NaN,<NA>
...,...,...,...,...,...,...,...
2028,2017-12-25,NaN,NaN,NaN,NaN,NaN,<NA>
2029,2017-12-26,175.99,176.63,177.00,174.67,4.0,tuesday
2030,2017-12-27,177.62,176.55,178.44,176.26,4.0,wednesday
2031,2017-12-28,177.92,177.95,178.94,177.68,4.0,thursday


In [ ]:
fb = fb.fillna(method='ffill')

In [ ]:
fb.head()

,d_fecha,c_cierre,c_apertura,c_maximo,c_minimo,v_quarter,v_weekday
0,2012-06-06,26.81,26.07,27.17,25.52,2.0,wednesday
1,2012-06-07,31.73,31.44,31.90,31.26,2.0,thursday
2,2012-06-08,21.92,21.39,22.15,21.30,2.0,friday
3,2012-06-09,21.92,21.39,22.15,21.30,2.0,friday
4,2012-06-10,21.92,21.39,22.15,21.30,2.0,friday


In [ ]:
fb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2033 entries, 0 to 2032
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   d_fecha     2033 non-null   datetime64[ns]
 1   c_cierre    2033 non-null   float64       
 2   c_apertura  2033 non-null   float64       
 3   c_maximo    2033 non-null   float64       
 4   c_minimo    2033 non-null   float64       
 5   v_quarter   2033 non-null   float64       
 6   v_weekday   2033 non-null   string        
dtypes: datetime64[ns](1), float64(5), string(1)
memory usage: 191.6 KB


In [ ]:
fb["v_weekday"]=fb["d_fecha"].map(lambda x:x.strftime('%A'))
fb = fb.astype(dtype={'v_weekday': 'string'})
fb = fb.astype(dtype={'v_quarter': 'int'})

In [ ]:
fb.head()

,d_fecha,c_cierre,c_apertura,c_maximo,c_minimo,v_quarter,v_weekday
0,2012-06-06,26.81,26.07,27.17,25.52,2,Wednesday
1,2012-06-07,31.73,31.44,31.90,31.26,2,Thursday
2,2012-06-08,21.92,21.39,22.15,21.30,2,Friday
3,2012-06-09,21.92,21.39,22.15,21.30,2,Saturday
4,2012-06-10,21.92,21.39,22.15,21.30,2,Sunday


In [ ]:
fb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2033 entries, 0 to 2032
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   d_fecha     2033 non-null   datetime64[ns]
 1   c_cierre    2033 non-null   float64       
 2   c_apertura  2033 non-null   float64       
 3   c_maximo    2033 non-null   float64       
 4   c_minimo    2033 non-null   float64       
 5   v_quarter   2033 non-null   int64         
 6   v_weekday   2033 non-null   string        
dtypes: datetime64[ns](1), float64(4), int64(1), string(1)
memory usage: 191.6 KB


In [ ]:
fb['v_weekday'] = fb['v_weekday'].str.lower()

In [ ]:
fb = fb.drop(fb[fb['v_weekday'] == 'saturday'].index)
fb = fb.drop(fb[fb['v_weekday'] == 'sunday'].index)
fb.reset_index(drop=True,inplace=True)

In [ ]:
fb.head()

,d_fecha,c_cierre,c_apertura,c_maximo,c_minimo,v_quarter,v_weekday
0,2012-06-06,26.81,26.07,27.17,25.52,2,wednesday
1,2012-06-07,31.73,31.44,31.90,31.26,2,thursday
2,2012-06-08,21.92,21.39,22.15,21.30,2,friday
3,2012-06-11,21.17,21.24,21.37,20.99,2,monday
4,2012-06-12,26.97,27.68,27.75,26.82,2,tuesday


In [ ]:
fb['d_fecha'].describe()

count                    1453
unique                   1453
top       2012-06-06 00:00:00
freq                        1
first     2012-06-06 00:00:00
last      2017-12-29 00:00:00
Name: d_fecha, dtype: object

In [ ]:
fb["d_fecha"].diff().describe()

count                         1452
mean     1 days 09:35:12.396694214
std      0 days 19:11:48.073539184
min                1 days 00:00:00
25%                1 days 00:00:00
50%                1 days 00:00:00
75%                1 days 00:00:00
max                3 days 00:00:00
Name: d_fecha, dtype: object

In [ ]:
fb.shape

(1453, 7)

Listo, hemos limpiado, ordenado y acompletado los registros de los precios de las acciones de Facebook, ahora tenemos registros diarios de los días entre semana.

#### Orden

Verificamos el orden por fecha

In [ ]:
fb["d_fecha"]=pd.to_datetime(fb["d_fecha"])
fb=fb.sort_values(by=["d_fecha"])
fb.reset_index(drop=True,inplace=True)

In [ ]:
fb.head(100)

,d_fecha,c_cierre,c_apertura,c_maximo,c_minimo,v_quarter,v_weekday
0,2012-06-06,26.81,26.07,27.17,25.52,2,wednesday
1,2012-06-07,31.73,31.44,31.90,31.26,2,thursday
2,2012-06-08,21.92,21.39,22.15,21.30,2,friday
3,2012-06-11,21.17,21.24,21.37,20.99,2,monday
4,2012-06-12,26.97,27.68,27.75,26.82,2,tuesday
...,...,...,...,...,...,...,...
95,2012-10-17,19.88,19.50,20.48,19.37,4,wednesday
96,2012-10-18,18.98,19.70,19.79,18.89,4,thursday
97,2012-10-19,19.00,19.00,19.06,18.80,4,friday
98,2012-10-22,19.32,19.20,19.43,19.05,4,monday


#### ¿Cada cuantos días se recopiló la información?

Nuestros datos ahora se presentan diarios de lunes a viernes.

In [ ]:
fb["d_fecha"].diff().describe()

count                         1452
mean     1 days 09:35:12.396694214
std      0 days 19:11:48.073539184
min                1 days 00:00:00
25%                1 days 00:00:00
50%                1 days 00:00:00
75%                1 days 00:00:00
max                3 days 00:00:00
Name: d_fecha, dtype: object

In [ ]:
fb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1453 entries, 0 to 1452
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   d_fecha     1453 non-null   datetime64[ns]
 1   c_cierre    1453 non-null   float64       
 2   c_apertura  1453 non-null   float64       
 3   c_maximo    1453 non-null   float64       
 4   c_minimo    1453 non-null   float64       
 5   v_quarter   1453 non-null   int64         
 6   v_weekday   1453 non-null   string        
dtypes: datetime64[ns](1), float64(4), int64(1), string(1)
memory usage: 79.6 KB


#### Ingeniería de variables